In [ ]:
# ******************************************
# Ionel Rodriguez 8-764-368
# Modelos Predictivos
# ******************************************

import pandas
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy import stats
import warnings
import math
import sys

from statsmodels.tsa import holtwinters
from sklearn.linear_model import LinearRegression, PoissonRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics as skm
from math import sqrt, pow
import itertools

sys.path.append('./.')
from lib import comun

warnings.filterwarnings('ignore')
df_main = comun.load_base_dataframe()

In [ ]:
def tabla_suavizacion_exponencial(df, alpha):
	df = df.copy()
	dt = df.Dt[1:]
	
	Lt = [np.mean(dt)]
	for i,x in enumerate(dt,1): Lt.append(alpha*x + (1-alpha)*Lt[i-1])
	df['Lt'] = Lt
	df['Ft'] = [np.nan] + list(df.Lt[0:-1])

	df['Et'] = df.Ft - df.Dt
	df['At'] = df.Et.abs()
	df['MSE'] = df.index.map(lambda x: np.sum(np.power(df.Et[0:x+1], 2))/x)
	df['MAD'] = df.index.map(lambda x: np.sum(df.At[0:x+1])/x)
	df['Error'] = df.At/df.Dt * 100
	df['MAPE'] = df.index.map(lambda x: np.mean(df.Error[0:x+1]))
	df['Tst'] = df.index.map(lambda x: np.sum(df.Et[0:x+1])/df.MAD[x])

	return df

def tabla_suavizacion_holt(df, alpha, beta):
	df = df.copy()
	dt = df.Dt[1:]

	lr = stats.linregress(df.index[1:], dt)
	Lt = [lr.intercept]
	Tt = [lr.slope]

	for i,x in enumerate(dt,1):
		Lt.append(alpha*x + (1-alpha)*(Lt[i-1] + Tt[i-1]))
		Tt.append(beta*(Lt[i]-Lt[i-1]) + (1-beta)*Tt[i-1])

	df['Lt'] = Lt
	df['Tt'] = Tt
	df['FT'] = [np.nan] + list(df.Tt + df.Lt)[0:-1]
	
	df['Et'] = df.FT - df.Dt
	df['At'] = df.Et.abs()
	df['MSE'] = df.index.map(lambda x: np.sum(np.power(df.Et[0:x+1], 2))/x)
	df['MAD'] = df.index.map(lambda x: np.sum(df.At[0:x+1])/x)
	df['Error'] = df.At/df.Dt * 100
	df['MAPE'] = df.index.map(lambda x: np.mean(df.Error[0:x+1]))
	df['Tst'] = df.index.map(lambda x: np.sum(df.Et[0:x+1])/df.MAD[x])

	return df

for idx, m in enumerate(df_main.modelo.cat.categories):
	dt = list(df_main[df_main.modelo_id == idx].cantidad)
	periodos = 12

	#dt = [8000, 13000, 23000, 34000, 10000, 18000, 23000, 38000, 12000, 13000, 32000, 41000]
	#periodos = 4

	len_dt = len(dt)

	df = pandas.DataFrame({'Dt': [np.nan] + dt})
	df.index = np.arange(len_dt+1)

	desestacionalizada0 = [(dt[i-2] + dt[i+2] + 2*sum(dt[i-1:i+2]))/(len_dt-4) for i in range(2, len_dt-2)]
	lr = stats.linregress(range(3, len(desestacionalizada0)+3), desestacionalizada0)
	
	desestacionalizada = df.index.map(lambda x: x*lr.slope + lr.intercept if x > 0 else np.nan)
	factor_estacional = df.index.map(lambda x: df.Dt.iloc[x]/desestacionalizada[x])
	factores = [np.mean([factor_estacional[i+j+1] for j in range(0,len_dt, periodos)]) for i in range(periodos)]
	
	# suavizacion exponencial
	suav = tabla_suavizacion_exponencial(df, 0.1)
	holt = tabla_suavizacion_holt(df, 0.1, 0.2)
	
	#display(suav)
	display(holt)

	break
	
